<img src="valorumdata.svg" height="100px" width="200px" style="float: right;"/>

# Pandas: Cleaning Data
---

**Prerequisites**

- Intro
- Boolean selection
- Indexing

**Outcomes**

- Be able to use string methods to clean data that comes in as a string
- Be able to drop missing data
- Use cleaning methods to prepare and analyze a real dataset

**Data**

- Item information from about 3,000 Chipotle meals from about 1,800 Grubhub orders

---
    


In [1]:
import pandas as pd
import numpy as np
import valorum

## Cleaning data

A significant portion of data work often does not actually involve analyzing data, rather, it often involves preparing the data for analysis

This non-analysis work is often called "data cleaning"

Pandas exposes very powerful tools for doing data cleaning which we will demonstrate using the following dataset

In [2]:
df = pd.DataFrame({"numbers": ["#23", "#24", "#18", "#14", "#12", "#10", "#35"],
                   "nums": ["23", "24", "18", "14", np.nan, "XYZ", "35"],
                   "colors": ["green", "red", "yellow", "orange", "purple", "blue", "pink"],
                   "other_column": [0, 1, 0, 2, 1, 0, 2]})
df

colors numbers nums  other_column
0   green     #23   23             0
1     red     #24   24             1
2  yellow     #18   18             0
3  orange     #14   14             2
4  purple     #12  NaN             1
5    blue     #10  XYZ             0
6    pink     #35   35             2

What would happen if we wanted to try and compute the mean of `numbers`?

In [3]:
df["numbers"].mean()

TypeError: Could not convert #23#24#18#14#12#10#35 to numeric

It throws an error.

Can you figure out why?

Hint: When looking at error messages, it helps to start at the very bottom.

The final error says, `TypeError: Could not convert #23#24... to numeric`

**Exercise**: Convert the string below into a number

In [4]:
c2n = "#39"

## String methods

One possible approach to the previous exercise was to remove the `#` by using the `replace` string method

One way to make this change to every element of a column would be to loop through all elements of the column and apply the desired string methods...

In [5]:
%%time

# Iterate over all rows
for row in df.iterrows():

    # `iterrows` method produces a tuple with two elements...
    # The first element is an index and second is a series with the data from that row
    index_value, column_values = row

    # Apply string method
    clean_number = column_values["numbers"].replace("#", "")

    # The `at` method is very similar to the `loc` method, but it is specialized
    # for accessing single elements at a time... We wanted to use it here to give
    # the loop the best chance to beat a faster method we show you next
    df.at[index_value, "numbers_loop"] = clean_number


CPU times: user 4.61 ms, sys: 0 ns, total: 4.61 ms
Wall time: 4.41 ms


While this is fast for a small dataset like this, it turns out to be slow for larger datasets

There is a _significantly_ faster way to apply a string method to an entire column of data (see appendix at the end of the notebook)

Any method that follows a `.str` on a Series is applied to the entire column

For example, to create a Series that replaces all of the `#` with nothing, we would write `df["numbers"].str.replace("#", "")`

In [6]:
%%time

# ~2x faster than loop... However speed gain increases with size of DataFrame. The
# speedup can be in the ballpark of ~100-500x faster for big DataFrames
df["numbers_str"] = df["numbers"].str.replace("#", "")

CPU times: user 1.45 ms, sys: 74 µs, total: 1.52 ms
Wall time: 1.42 ms


We can use this notation to access almost any string method that works on normal strings (see the [official documentation](https://pandas.pydata.org/pandas-docs/stable/text.html) for more information)

In [7]:
df["colors"].str.contains("p")

0    False
1    False
2    False
3    False
4     True
5    False
6     True
Name: colors, dtype: bool

In [8]:
df["colors"].str.capitalize()

0     Green
1       Red
2    Yellow
3    Orange
4    Purple
5      Blue
6      Pink
Name: colors, dtype: object

**Exercise**: Make a new column called `colors_upper` that contains the elements of `colors` with all uppercase letters.

## Type conversions

In our example above, even after we have removed the `"#"`, the `dtype` of `numbers` shows that pandas still treats it as a string

In [9]:
df.dtypes

colors          object
numbers         object
nums            object
other_column     int64
numbers_loop    object
numbers_str     object
dtype: object

We still must convert it to a number

The best way to do this is using the `pd.to_numeric` function.

This method attempts to convert whatever is stored in a Series into numeric values.

For example, after having removed the `"#"` from the numbers in column `"numbers"`, they are ready to be converted to numbers.

In [10]:
df["numbers_numeric"] = pd.to_numeric(df["numbers_str"])

In [11]:
df.dtypes

colors             object
numbers            object
nums               object
other_column        int64
numbers_loop       object
numbers_str        object
numbers_numeric     int64
dtype: object

In [12]:
df.head()

colors numbers nums  other_column numbers_loop numbers_str  numbers_numeric
0   green     #23   23             0           23          23               23
1     red     #24   24             1           24          24               24
2  yellow     #18   18             0           18          18               18
3  orange     #14   14             2           14          14               14
4  purple     #12  NaN             1           12          12               12

There is support for converting to other types well

In a later lecture we will discuss converting columns into a `datetime` type

We can also try and "force" conversions using the `astype` method... For example, we could convert the `"numbers_numeric"` column back into strings using `df["numbers_numeric"].astype(str)`

In [13]:
df["numbers_numeric"].astype(str)

0    23
1    24
2    18
3    14
4    12
5    10
6    35
Name: numbers_numeric, dtype: object


**Exercise**: Convert the column `"nums"` to a numeric type and save it to the DataFrame as `"nums_tonumeric"`. Notice that there is a missing value and a value that is not a number. Look at the documentation for `pd.to_numeric` and think about how to overcome this.

Discuss with your neighbor why this could be a bad idea of used without knowing what your data looks like (Think about what happens when you apply it to the `"numbers"` column before replacing the `"#"`)

## Missing data

Most datasets contain at least some missing data

In our example, we are missing an element from the `"nums"` column.

In [14]:
df

colors numbers nums  other_column numbers_loop numbers_str  numbers_numeric
0   green     #23   23             0           23          23               23
1     red     #24   24             1           24          24               24
2  yellow     #18   18             0           18          18               18
3  orange     #14   14             2           14          14               14
4  purple     #12  NaN             1           12          12               12
5    blue     #10  XYZ             0           10          10               10
6    pink     #35   35             2           35          35               35

We can find missing data by using the `isnull` method.

In [15]:
df.isnull()

colors  numbers   nums  other_column  numbers_loop  numbers_str  \
0   False    False  False         False         False        False   
1   False    False  False         False         False        False   
2   False    False  False         False         False        False   
3   False    False  False         False         False        False   
4   False    False   True         False         False        False   
5   False    False  False         False         False        False   
6   False    False  False         False         False        False   

   numbers_numeric  
0            False  
1            False  
2            False  
3            False  
4            False  
5            False  
6            False

We might want to know whether particular rows have any missing data and can explore this with the `any` concept

In [16]:
df.isnull().any(axis=0)

colors             False
numbers            False
nums                True
other_column       False
numbers_loop       False
numbers_str        False
numbers_numeric    False
dtype: bool

In [17]:
df.isnull().any(axis=1)

0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

There are many approaches to dealing with missing data. Two that are commonly used because of their simplicity are

* Exclusion: Ignore any data that is missing
* Imputation: Compute "predicted" values for the data that is missing

For the advantages and disadvantages of these (and other) approaches, consider reading the [Wikipedia article](https://en.wikipedia.org/wiki/Missing_data) or taking a statistics course on the topic.

Pandas allows us the option to exclude missing data. Imputing values or doing analysis while including missing data is beyond its scope.

In [18]:
df.dropna()

colors numbers nums  other_column numbers_loop numbers_str  numbers_numeric
0   green     #23   23             0           23          23               23
1     red     #24   24             1           24          24               24
2  yellow     #18   18             0           18          18               18
3  orange     #14   14             2           14          14               14
5    blue     #10  XYZ             0           10          10               10
6    pink     #35   35             2           35          35               35

We will see more examples of dealing with missing data in future notebooks

## Case Study

We will now use data from an [article](https://www.nytimes.com/interactive/2015/02/17/upshot/what-do-people-actually-order-at-chipotle.html) written by The Upshot at the NYTimes.

This data has order information from almost 2,000 Chipotle orders and includes information on what was ordered and how much it cost.

In [19]:
chipotle = valorum.data.load("chipotle_raw")
chipotle.head()

order_id  quantity                              item_name  \
0         1         1           Chips and Fresh Tomato Salsa   
1         1         1                                   Izze   
2         1         1                       Nantucket Nectar   
3         1         1  Chips and Tomatillo-Green Chili Salsa   
4         2         2                           Chicken Bowl   

                                  choice_description item_price  
0                                                NaN     $2.39   
1                                       [Clementine]     $3.39   
2                                            [Apple]     $3.39   
3                                                NaN     $2.39   
4  [Tomatillo-Red Chili Salsa (Hot), [Black Beans...    $16.98

**Exercise**: We'd like you to use this data to answer the following questions

- What is the average price of an item with chicken?
- What is the average price of an item with steak?
- Did chicken or steak produce more revenue (total)?
- How many missing items are there in this dataset? How many missing items in each column?

Hint: before you will be able to do any of these things you will need to make sure the `item_price` column has a numeric `dtype` (probably float)

## Appendix: performance of  `.str` methods

Let's repeat the "remove the `#`" example from above, but this time on a much larger dataset


In [20]:
import numpy as np
test = pd.DataFrame({"answer": np.round(100*np.random.rand(100000), 2)})
test["strings"] = test["answer"].astype(str) + "%"
test.head()

answer strings
0    8.41   8.41%
1   45.14  45.14%
2   62.01  62.01%
3    1.05   1.05%
4   77.36  77.36%

In [21]:
%%time

for row in test.iterrows():
    index_value, column_values = row
    clean_number = column_values["strings"].replace("%", "")
    test.at[index_value, "numbers_loop"] = clean_number

CPU times: user 7.71 s, sys: 1.24 ms, total: 7.71 s
Wall time: 7.71 s


In [22]:
%%time
test["numbers_str_method"] = test["strings"].str.replace("%", "")

CPU times: user 52.5 ms, sys: 0 ns, total: 52.5 ms
Wall time: 51.3 ms


In [23]:
test["numbers_str_method"].equals(test["numbers_loop"])

True

We got the exact same result in a fraction of the time!